In [5]:
#!/usr/bin/python3

import numpy as np
import modern_robotics as mr
import math

'''
Course 2 Project
'''

def IKinBodyIterates(Blist, M, T, thetalist0, eomg, ev,
        filename='iteration.csv'):
    """
    this function is modified function of IKinBody from modern_robotics package
    the function consist of
    
    IKinBody
    printIterationsummary  (print all iteration information)
    writeJointValues  (Writes iteration joint values to CSV file.)
    
    
    
    Computes inverse kinematics in the body frame for an open chain robot
    outputting information at each iteration.
    
    :param Blist: The joint screw axes in the end-effector frame when the
                  manipulator is at the home position, in the format of a
                  matrix with axes as the columns
    :param M: The home configuration of the end-effector
    :param T: The desired end-effector configuration Tsd
    :param thetalist0: An initial guess of joint angles that are close to
                       satisfying Tsd
    :param eomg: A small positive tolerance on the end-effector orientation
                 error. The returned joint angles must give an end-effector
                 orientation error less than eomg
    :param ev: A small positive tolerance on the end-effector linear position
               error. The returned joint angles must give an end-effector
               position error less than ev
    :param filename: The CSV filename to which joint value for each iteration
                     are output. Default is 'iterates.csv'.
    :return thetalist: Joint angles that achieve T within the specified
                       tolerances,
    :return success: A logical value where TRUE means that the function found
                     a solution and FALSE means that it ran through the set
                     number of maximum iterations without finding a solution
                     within the tolerances eomg and ev.
    Uses an iterative Newton-Raphson root-finding method.
    The maximum number of iterations before the algorithm is terminated has
    been hardcoded in as a variable called maxiterations. It is set to 20 at
    the start of the function, but can be changed if needed.
    
    Example Input:
        Blist = np.array([[0, 0, -1, 2, 0,   0],
                          [0, 0,  0, 0, 1,   0],
                          [0, 0,  1, 0, 0, 0.1]]).T
        M = np.array([[-1, 0,  0, 0],
                      [ 0, 1,  0, 6],
                      [ 0, 0, -1, 2],
                      [ 0, 0,  0, 1]])
        T = np.array([[0, 1,  0,     -5],
                      [1, 0,  0,      4],
                      [0, 0, -1, 1.6858],
                      [0, 0,  0,      1]])
        thetalist0 = np.array([1.5, 2.5, 3])
        eomg = 0.01
        ev = 0.001
    Output:
        (np.array([1.57073819, 2.999667, 3.14153913]), True)
    Writes information about each iteration to console and writes joint values
    for each iteration to CSV file.
    """

    '''
  printIterationSummary  Prints iteration information to console.
    '''
    def printIterationSummary(i, thetalist, Tb, Vb, errorAngular, errorLinear):
        print('Iteration %d' %i)
        print('joint vector: %s' %thetalist)
        print('SE(3) end effector configuration:','\n','%s' %Tb)
        print('twist error: %s' %Vb)
        print('angular error magnitude: %s' %errorAngular)
        print('linear error magnitude: %s' %errorLinear)
        print('')

        
    '''
  writeJointValues  Writes iteration joint values to CSV file.
    '''
    def writeJointValues(file, thetalist):
        for i in range(len(thetalist)):
            file.write('%s' %thetalist[i])
            if i < len(thetalist) - 1:
                file.write(',')
        file.write('\n')

    file = open(filename, 'w')

    thetalist = np.array(thetalist0).copy()
    i = 0
    maxiterations = 40
    error = True
    Vb = np.zeros(6) #this us= twist error
    while error and i < maxiterations:
        thetalist = thetalist + np.dot(np.linalg.pinv(mr.JacobianBody(Blist, thetalist)), Vb)
        Tb = mr.FKinBody(M, Blist, thetalist) # current end effector configuration
        Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(Tb), T)))
        errorAngular = np.linalg.norm([Vb[0], Vb[1], Vb[2]]) # twist angular error
        errorLinear = np.linalg.norm([Vb[3], Vb[4], Vb[5]]) # twist linear error
        error = (errorAngular > eomg) or (errorLinear > ev)
        printIterationSummary(i, thetalist, Tb, Vb, errorAngular, errorLinear)
        writeJointValues(file, thetalist)
        i += 1
    file.close()

    return (thetalist, not error)

def example4_5():
    W1 = 0.109 # meters
    W2 = 0.082
    L1 = 0.425
    L2 = 0.392
    H1 = 0.089
    H2 = 0.095

    M = np.array([[-1, 0, 0, L1 + L2],
                  [0, 0, 1, W1 + W2],
                  [0, 1, 0, H1 - H2],
                  [0, 0, 0, 1]])
    
    Blist = np.array([[       0,        0,   0,  0,   0,   0],
                      [       0,        0,   0,  0,  -1,   0],
                      [       1,        1,   1,  1,   0,   1],
                      [ W1 + W2,       H2,  H2, H2, -W2,   0],
                      [       0, -(L1+L2), -L2,  0,   0,   0],
                      [ L1 + L2,        0,   0,  0,   0,   0]]) 
    
    T = np.array([[0, 1, 0, -0.5],
                  [0, 0, -1, 0.1],
                  [-1, 0, 0, 0.1],
                  [0, 0, 0, 1]])
    eomg = 0.001
    ev = 0.0001
    thetalist0 = np.array([0, 4, 5, 4, 3, 5])
    
    IKinBodyIterates(Blist, M, T, thetalist0, eomg, ev)

"""" 
testing IKinBodyIteration function
def test():
    Blist = np.array([[ 0, 0, 0   ],
                      [ 0, 0, 0   ],
                      [-1, 0, 1   ],
                      [ 2, 0, 0   ],
                      [ 0, 1, 0   ],
                      [ 0, 0, 0.1 ]])

    M = np.array([[-1, 0,  0, 0],
                  [ 0, 1,  0, 6],
                  [ 0, 0, -1, 2],
                  [ 0, 0,  0, 1]])
    
    T = np.array([[0, 1,  0,     -5],
                  [1, 0,  0,      4],
                  [0, 0, -1, 1.6858],
                  [0, 0,  0,      1]])
    thetalist0 = np.array([1.5, 2.5, 3])
    eomg = 0.01
    ev = 0.001
    thetalist, success = IKinBodyIteration(Blist, M, T, thetalist0, eomg, ev)
    
    np.testing.assert_almost_equal(thetalist,
            np.array([1.57073819, 2.999667, 3.14153913]))
    assert(success == True)
    
    #thetalist0 = np.array([0, 5*math.pi/4, 3*math.pi/2, 3*math.pi/4, math.pi, 3*math.pi/2])
    #thetalist0 = np.array([-0.1, 4.2, 4.6, 3.9, 3.0, 4.7])
"""""

if __name__ == '__main__':
    example4_5()
    


Iteration 0
joint vector: [0. 4. 5. 4. 3. 5.]
SE(3) end effector configuration: 
 [[-0.14807605  0.98064999  0.1280589  -0.66437664]
 [ 0.04003041  0.1353234  -0.9899925   0.02782062]
 [-0.9881655  -0.14146793 -0.05929398  0.15802106]
 [ 0.          0.          0.          1.        ]]
twist error: [ 0.13260277 -0.05000597  0.14577416  0.04752267  0.1728707  -0.05971579]
angular error magnitude: 0.20330813181817564
linear error magnitude: 0.18896735081127378

Iteration 1
joint vector: [0.10532382 4.11346965 4.17260096 4.53535195 3.14134082 5.07325362]
SE(3) end effector configuration: 
 [[ 4.88866617e-04  9.99999853e-01  2.35659675e-04 -4.41078945e-01]
 [ 8.89192846e-05  2.35616233e-04 -9.99999968e-01  1.13049566e-01]
 [-9.99999877e-01  4.88887556e-04 -8.88040866e-05  1.24045414e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
twist error: [ 2.35637966e-04 -8.88616901e-05 -4.88877111e-04  2.40304316e-02
 -5.89284685e-02  1.30436921e-02]
angular error magnitude: 